### 9.1 Wrapper

In [1]:
import time
from functools import wraps
from inspect import signature

In [2]:
def timeit(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [3]:
@timeit
def countdown(n:int):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1
        
countdown(100000)
countdown(1000)
        

countdown 0.011677980422973633
countdown 6.29425048828125e-05


In [4]:
countdown.__name__, countdown.__doc__, countdown.__annotations__, countdown.__wrapped__

('countdown',
 '\n    Counts down\n    ',
 {'n': int},
 <function __main__.countdown(n: int)>)

In [5]:
print(signature(countdown))

(n: int)


### 9.4 Decorator with arguments

In [6]:
import logging
logging.basicConfig(level=logging.WARNING)

In [7]:
from functools import partial

# Utility decorator to attach a func as an attribute of an obj
def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj) # This is the wrapper around the func -> attach_wrapper(obj)(func)
    setattr(obj, func.__name__, func)
    return func

In [8]:
def logged(level, name=None, message=None):
    
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
    
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel
            
        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg
            
        return wrapper
        
    return decorate

In [9]:
@logged(logging.WARNING)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

In [10]:
spam()
add(3, 4)

CRITICAL:example:spam


Spam!


7

In [11]:
spam.set_level(logging.ERROR)
spam.set_message('Custom msg')
spam()

ERROR:example:Custom msg


Spam!


### 9.7 Type Assertion

In [12]:
def spam(x, y, z=42):
    pass

In [13]:
sig = signature(spam)
print(sig)

(x, y, z=42)


In [14]:
sig.parameters

mappingproxy({'x': <Parameter "x">,
              'y': <Parameter "y">,
              'z': <Parameter "z=42">})

In [15]:
sig.parameters['z'].name, sig.parameters['z'].default, sig.parameters['z'].kind

('z', 42, <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>)

In [16]:
bound_types = sig.bind_partial(int, z=int)
bound_types.arguments

OrderedDict([('x', int), ('z', int)])

In [17]:
def typeassert(*ty_args, **ty_kwargs):
    
    def decorate(func):
        sig = signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments
        
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs).arguments
            # Enforce type assertions
            for name, value in bound_values.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError('Argument {} must be of type {}'.format(name, bound_types[name]))
            return func(*args, **kwargs)
        return wrapper
    return decorate
            
            

In [18]:
@typeassert(int, int)
def add(x, y=4):
    return x + y

@typeassert(y=str)
def printer(x, y, z):
    print(x, y, z)

print('2 + 3 = {}'.format(add(2, 3)))
printer(1, 'world', 3)
try:
    add('bat', 'man')
except TypeError as err:
    print('Error: {}'.format(err))
    
try:
    printer(1, 2, 3)
except TypeError as err:
    print('Error: {}'.format(err))

2 + 3 = 5
1 world 3
Error: Argument x must be of type <class 'int'>
Error: Argument y must be of type <class 'str'>
